In [1]:
import transformers
import torch
import torchvision

from tqdm import tqdm
from PIL import Image

import pandas as pd
import numpy as np

from torchinfo import summary
import os
import glob

import tokenizers
import itertools

import random
import math
import copy
from timm.scheduler import CosineLRScheduler

import wandb 
import nltk
nltk.download('stopwords')

# from utils.RegVLM import RegVLM
from utils.hog import HOGLayer

from utils.dataset import pretrain_dataset
from utils.mim_utils import create_masked_image

device = 'cuda:0'


DEVICE = torch.device(device) if torch.cuda.is_available() else torch.device('cpu')
model_name = 'ViT-S BERT-S (fixed everything)'


MU = 1.
NU = 1.

NUM_WORKERS = 8
torch.set_num_threads(12)


id = wandb.util.generate_id()
wandb.login()

# set earlier ID
# id = '6hcguuhh'


algo = f'RegVLM (no MLM and MIM Reg) - {MU} - {NU}'
print(id)

/home/ml/miniconda3/envs/py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/ml/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: madhava20217. Use `wandb login --relogin` to force relogin


xjbbtpc2


In [2]:
DATASET_JSON = 'Jsons/flickr30k_train.json'
MODEL_SAVE_PATH = f'Models/{model_name}/{algo}/checkpoint'

lr = 2.5e-4
init_lr = 1e-6
min_lr = 1e-5
decay = 0.01
beta1 = 0.9
beta2 = 0.999

warmup_epochs = 3
EPOCHS = 12

BATCH_SIZE = 96

MASKING_RATIO_IMG = 0.75
MASKING_RATIO_TXT = 0.25

HOG_BINS = 9


ALPHA = 0.995               # EWMA


n_layers = 2

if os.path.exists(os.path.dirname(MODEL_SAVE_PATH)) == False:
    os.makedirs(os.path.dirname(MODEL_SAVE_PATH))

In [3]:
DIMENSION = 224

MAX_LEN = 30

# ViT config
tokenizer = transformers.AutoTokenizer.from_pretrained("prajjwal1/bert-small")

In [4]:
import torchvision.transforms.v2 as v2

## image transforms
img_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.int8, scale = True),
    v2.Resize(size = (DIMENSION, DIMENSION), antialias = False),
    v2.RandAugment(),
    # v2.RandomVerticalFlip(),
    # v2.RandomHorizontalFlip(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(
        mean = [0.5, 0.5, 0.5],
        std =  [0.5, 0.5, 0.5]
    )
])

In [5]:
dataset = pretrain_dataset(
               ann_file = [DATASET_JSON],
               transform = img_transform,
               tokenizer = tokenizer,
               max_words = MAX_LEN,
               input_size = DIMENSION,
               mask_patch_size = 32,
               model_patch_size = 16,
               masking_ratio = MASKING_RATIO_IMG,
               txt_masking_ratio = MASKING_RATIO_TXT,
               mask_token = tokenizer.mask_token,
               mask_token_id = tokenizer.mask_token_id,
               max_length = MAX_LEN + 5,
               )

In [6]:
# dataloader

pretrain_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size = BATCH_SIZE,
    pin_memory = True,
    num_workers = NUM_WORKERS,
    shuffle = True
)

In [7]:
def deleteEncodingLayers(model, num_layers_to_keep):  # must pass in the full bert model
    oldModuleList = model.encoder.layer
    newModuleList = torch.nn.ModuleList()

    # Now iterate over all layers, only keepign only the relevant layers.
    for i in range(0, num_layers_to_keep):
        newModuleList.append(oldModuleList[i])

    # create a copy of the model, modify it with the new list, and return
    copyOfModel = copy.deepcopy(model)
    copyOfModel.encoder.layer = newModuleList

    return copyOfModel

def deleteLaterEncodingLayers(model, num_layers_to_keep):  # must pass in the full bert model
    oldModuleList = model.encoder.layer
    newModuleList = torch.nn.ModuleList()

    # Now iterate over all layers, only keepign only the relevant layers.
    for i in range(num_layers_to_keep, 0, -1):
        newModuleList.append(oldModuleList[-i])

    # create a copy of the model, modify it with the new list, and return
    copyOfModel = copy.deepcopy(model)
    copyOfModel.encoder.layer = newModuleList

    return copyOfModel

class RegVLM_Mixer(torch.nn.Module):
    def __init__(self, base_bert,
                 n_layers = 2,
                 n_visual_tokens = 196,
                 vision_embedding_dim = 384,
                 emb_dims = 512,
                 cls_token_id = 101):
        # prepare decoder
        super().__init__()
        self.cls_token_id = cls_token_id
        self.embedding_module = base_bert.embeddings
        self.n_visual_tokens = n_visual_tokens
        self.vision_emb_dim = vision_embedding_dim
        self.base_model = deleteLaterEncodingLayers(base_bert, n_layers).encoder
        
        self.pooler = torch.nn.AdaptiveAvgPool1d(1)
        self.emb_dimension = emb_dims
        
        if self.vision_emb_dim == self.emb_dimension:
            self.dimension_caster = torch.nn.Identity()
        else:
            self.dimension_caster = torch.nn.Linear(self.vision_emb_dim, self.emb_dimension, bias = False)  # no bias here
        
        
    def forward(self, vision_embedding, text_embedding, text_attn_mask):
        # assert len(vision_embedding) == len(text_embedding)
        n_batch = len(vision_embedding)
        
        cls_emb = self.embedding_module(torch.tensor([[self.cls_token_id]]*n_batch, 
                                                     device = vision_embedding.device),
                                        torch.tensor([[1]]*n_batch,
                                                     device = vision_embedding.device))
        
        # normalize dimensions
        new_vision_emb = self.dimension_caster(vision_embedding[:, 1:, :])   # remove cls token here
        
        # concatenate
        concatenated_emb = torch.cat([cls_emb, new_vision_emb, text_embedding], dim = 1)
        
        # create attention mask
        vision_attention_mask = torch.ones(n_batch, self.n_visual_tokens + 1).to(text_attn_mask.device) # add a cls token here
        attn_mask = torch.cat([vision_attention_mask, text_attn_mask], dim = 1)
        
        attn_mask = attn_mask[:, None, None, :]
        
        # forward
        return self.base_model(concatenated_emb, attn_mask)

In [8]:
class VicVLM(torch.nn.Module):
    def __init__(self,
                 vit,
                 bert,
                 vit_num_patches = 196,
                 vit_emb_dim = 384,
                 bert_emb_dim = 512,
                 bert_layers = 2,
                 vocab_size = 30522,
                 mask_token_id = 103,
                 cls_token_id = 101,
                 tau = None,
                 lamda = 25,
                 mu = 25,
                 nu = 1.,
                 eps = 1e-4):
       super().__init__()
       self.vit = vit.vit
       self.mim_reconstruction_joint = vit.decoder
       self.mim_reconstruction = copy.deepcopy(vit.decoder)
       self.bert = bert.base_model
       self.bert = deleteEncodingLayers(self.bert.base_model, bert_layers)
       self.fusion = RegVLM_Mixer(bert.base_model,
                              n_layers = bert_layers,
                              n_visual_tokens=vit_num_patches,
                              vision_embedding_dim=vit_emb_dim,
                              emb_dims = bert_emb_dim,
                              cls_token_id = cls_token_id)
       
       self.lamda = lamda
       self.mew = mu
       self.nu = nu
       self.eps = eps
              
       # vit patches data
       self.vit_num_patches = vit_num_patches
       
       # vocab size
       self.vocab_size = vocab_size
       # mask token
       self.mask_token_id = mask_token_id
       
       # learnable temperature parameter
       self.tau = torch.nn.Parameter(torch.FloatTensor([0.07]))      # uniform in range 1 to 5
       if tau is not None:
           self.tau = torch.nn.Parameter(torch.FloatTensor([tau]))      # uniform in range 1 to 5
       
       self.tau.requires_grad = True

       # joint representation
       self.pooler = torch.nn.Sequential(
           torch.nn.AdaptiveAvgPool1d(1),
           torch.nn.Flatten()
       )
       self.img_proj = torch.nn.Linear(vit_emb_dim, min(vit_emb_dim, bert_emb_dim))
       self.txt_proj = torch.nn.Linear(bert_emb_dim, min(vit_emb_dim, bert_emb_dim))

       
       # masked representation modeling
       self.mrm_proj = torch.nn.Sequential(
            torch.nn.Linear(bert_emb_dim, bert_emb_dim),
            torch.nn.Tanh(),
       )
       
       # head for masked image modeling
       self.mim_proj = torch.nn.Sequential(
           torch.nn.Linear(bert_emb_dim, vit_emb_dim),
       )
        
       # head for masked language modeling
       self.mlm_head_joint = copy.deepcopy(bert.cls)
       self.mlm_head = bert.cls
       
       self.itc_head = torch.nn.Linear(bert_emb_dim, bert_emb_dim)
       
       self.itm__head = torch.nn.Sequential(
           torch.nn.Linear(bert_emb_dim, bert_emb_dim),
           torch.nn.LeakyReLU(),
           torch.nn.Linear(bert_emb_dim, 1)
       )
    
       
       
    def forward(self, image, text, attn_mask,
                masked_pos = None,
                masked_text = None,
                image_text_matching = False,
                retrieval = False,
                ):
        
        if retrieval is True:
            img_rep = self.vit(image)['last_hidden_state']
            txt_rep = self.bert(text, attn_mask)['last_hidden_state']
            joint_rep = self.fusion(img_rep, txt_rep, attn_mask)['last_hidden_state']
            
            
            # img_rep = self.img_proj(self.pooler(img_rep.transpose(1,2)))
            # txt_rep = self.txt_proj(self.pooler(txt_rep.transpose(1,2)))
            return img_rep, txt_rep, joint_rep, self.itm__head(joint_rep[:, 0, :])
        
        if image_text_matching == True:
            img_rep = self.vit(image)['last_hidden_state']
            txt_rep = self.bert(text, attn_mask)['last_hidden_state']
            joint_rep = self.fusion(img_rep, txt_rep, attn_mask)['last_hidden_state']
            
            return img_rep, txt_rep, joint_rep, self.itm__head(joint_rep[:, 0, :])
        
        else:
            # return mask_img-clean_txt, clean_img,-mask_txt, 
            img_rep = self.vit(image)['last_hidden_state']              # clean image
            txt_rep = self.bert(text, attn_mask)['last_hidden_state']   # clean text
            
            mask_img_rep = self.vit(image, bool_masked_pos = masked_pos)['last_hidden_state']
            mask_txt_rep = self.bert(masked_text, attn_mask)['last_hidden_state']
            
            # multimodal prediction
            c_img_m_txt = self.fusion(img_rep, mask_txt_rep, attn_mask)['last_hidden_state']
            m_img_c_txt = self.fusion(mask_img_rep, txt_rep, attn_mask)['last_hidden_state']
            
            # pure txt
            txt_prediction = self.mlm_head(mask_txt_rep)
            
            # pure fusion
            img_txt_joint = self.fusion(img_rep, txt_rep, attn_mask)['last_hidden_state']
        
            return (c_img_m_txt, m_img_c_txt, img_txt_joint, mask_img_rep, txt_prediction, img_rep, txt_rep)


    def get_mim_loss(self, img_rep, img, bool_masked_pos):
        sequence_output = img_rep

        # Reshape to (batch_size, num_channels, height, width)
        sequence_output = sequence_output[:, 1:]
        batch_size, sequence_length, num_channels = sequence_output.shape
        height = width = math.floor(sequence_length**0.5)
        sequence_output = sequence_output.permute(0, 2, 1).reshape(batch_size, num_channels, height, width)

        # Reconstruct pixel values
        reconstructed_pixel_values = self.mim_reconstruction(sequence_output)

        masked_im_loss = None
        if bool_masked_pos is not None:
            size = 224 // 16
            bool_masked_pos = bool_masked_pos.reshape(-1, size, size)
            mask = (
                bool_masked_pos.repeat_interleave(16, 1)
                .repeat_interleave(16, 2)
                .unsqueeze(1)
                .contiguous()
            )
            reconstruction_loss = torch.nn.functional.l1_loss(img, reconstructed_pixel_values, reduction="none")
            masked_im_loss = (reconstruction_loss * mask).sum() / (mask.sum() + 1e-5) / 3

        return masked_im_loss
    
    def get_joint_mim_loss(self, joint_rep, img, bool_masked_pos):
        joint_rep = joint_rep[:, :self.vit_num_patches+1, :]
        joint_rep = self.mim_proj(joint_rep)                    # bs x vit_npatch x vit dim
        sequence_output = joint_rep

        # Reshape to (batch_size, num_channels, height, width)
        sequence_output = sequence_output[:, 1:]
        batch_size, sequence_length, num_channels = sequence_output.shape
        height = width = math.floor(sequence_length**0.5)
        sequence_output = sequence_output.permute(0, 2, 1).reshape(batch_size, num_channels, height, width)

        # Reconstruct pixel values
        reconstructed_pixel_values = self.mim_reconstruction_joint(sequence_output)

        masked_im_loss_joint = None
        if bool_masked_pos is not None:
            size = 224 // 16
            bool_masked_pos = bool_masked_pos.reshape(-1, size, size)
            mask = (
                bool_masked_pos.repeat_interleave(16, 1)
                .repeat_interleave(16, 2)
                .unsqueeze(1)
                .contiguous()
            )
            reconstruction_loss = torch.nn.functional.l1_loss(img, reconstructed_pixel_values, reduction="none")
            masked_im_loss_joint = (reconstruction_loss * mask).sum() / (mask.sum() + 1e-5) / 3

        return masked_im_loss_joint
    
    
    def get_joint_mlm_loss(self, joint_reps, sen, masked_sen):
        embs = joint_reps[:, self.vit_num_patches+1:, :]
        scores = self.mlm_head_joint(torch.nn.functional.leaky_relu(embs))
        labels = torch.where(masked_sen == self.mask_token_id, sen, -100)
        loss = torch.nn.functional.cross_entropy(scores.view(-1, self.vocab_size), labels.view(-1), ignore_index=-100)
        return loss
    
    def get_mlm_loss(self, scores, sen, masked_sen):
        labels = torch.where(masked_sen == self.mask_token_id, sen, -100)
        loss = torch.nn.functional.cross_entropy(scores.view(-1, self.vocab_size), labels.view(-1), ignore_index=-100)
        
        return loss
    
    
    def get_itc_loss(self, img_feats, txt_feats):
        # Calculate similarity
        with torch.no_grad():
            self.tau.clamp_(0.001,0.5)

        # pool and flatten text and visual features obtained before fusion
        img_feats = self.img_proj(self.pooler(img_feats.transpose(1,2)))
        txt_feats = self.txt_proj(self.pooler(txt_feats.transpose(1,2)))
        
        
        sim = (img_feats@txt_feats.T)/self.tau
        # sim = torch.clip(sim, max = 1e4, min = 1e-4)
        self_mask = torch.eye(sim.shape[0], device=sim.device)
        
        loss_i2t = -torch.sum(torch.nn.functional.log_softmax(sim, dim = 1)*self_mask, dim = 1).mean()
        loss_t2i = -torch.sum(torch.nn.functional.log_softmax(sim.T, dim = 1)*self_mask, dim = 1).mean()

        return sim, (loss_i2t+loss_t2i)/2.0
    
    def get_samples(self, similarities):
        probs = torch.nn.functional.softmax(similarities, dim = 1) + 1e-8       # term to make nonnegative
        probs = probs.fill_diagonal_(0)         # eliminate full samples
        
        txt_indices = torch.multinomial(probs, num_samples=1, replacement=True).squeeze(1)
        img_indices = torch.multinomial(probs.T, num_samples=1, replacement=True).squeeze(1)
        
        return txt_indices, img_indices
    
    def get_vicreg_loss(self, features):
        #Source: https://arxiv.org/pdf/2105.04906.pdf
              
        # Calculate invariance -- not required for now as we're only using a part of vicreg
        # invariance_loss = torch.nn.functional.mse_loss(img_txt_feats, img_txt_feats_prime)

        # Calculate variance of img_txt_feats using torch.var
        features = torch.nn.functional.adaptive_avg_pool1d(features.transpose(1,2), 1).flatten(1)
        variance = torch.var(features, dim=0)
        std = torch.sqrt(variance + self.eps)
        std_loss = torch.mean(torch.relu(1 - std))

        # Calculate covariance of img_txt_feats using torch after centering the data
        centered_feats = features - torch.mean(features, dim=0)
        covariance = torch.mm(centered_feats.T, centered_feats) / (features.shape[0] - 1)        # Get off diagonal elements of covariance
        cov_loss = torch.sum(torch.pow(covariance - torch.diag(torch.diag(covariance)),2)) / (features.shape[1])

        return  + self.mew * std_loss + self.nu * cov_loss

In [9]:
vit_model = transformers.ViTForMaskedImageModeling.from_pretrained('WinKawaks/vit-small-patch16-224').to(DEVICE)
bert_model = transformers.BertForMaskedLM.from_pretrained("prajjwal1/bert-small")

online_network = RegVLM(
                    vit = vit_model,
                    bert = bert_model,
                    vit_num_patches= 196,
                    vit_emb_dim=384,
                    bert_emb_dim=512,
                    bert_layers=n_layers,
                    vocab_size=tokenizer.vocab_size,
                    mask_token_id= tokenizer.mask_token_id,
                    mu = MU,
                    nu = NU
                    # cls_token_id=tokenizer.cls_token_id
                ).train().to(DEVICE)

Some weights of ViTForMaskedImageModeling were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['decoder.0.bias', 'decoder.0.weight', 'vit.embeddings.mask_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
#optimiser
optim = torch.optim.AdamW(online_network.parameters(),
                          lr = lr,
                          weight_decay = decay,
                          betas = [0.9, 0.999],
                          )

epoch_steps = math.ceil(len(dataset)/BATCH_SIZE)
num_steps = int(EPOCHS * epoch_steps)
warmup_steps = int(warmup_epochs * epoch_steps)

lr_scheduler = CosineLRScheduler(
        optim,
        t_initial=num_steps,
        # t_mul=1.,
        lr_min=min_lr,
        warmup_lr_init = init_lr,
        warmup_t=warmup_steps,
        cycle_limit=1,
        t_in_epochs=False,
    )


# wandB init
wandb.init(
    id = id,# id,
    resume =  'allow',
    project = 'MAMO - Pretrain',
    name = 'RegVLM - ViT-S, BERT-S',

    config = {
        'architecture': model_name,
        'dataset':'ImageNet1K',
        'warmup_epochs': warmup_epochs,
        'epochs' : EPOCHS,
        'batch_size': BATCH_SIZE,
        'masking_ratio_img' : MASKING_RATIO_IMG,
        'masking_ratio_itxt' : MASKING_RATIO_TXT,
        'mask_patch_size': 196,
        'image_size' : DIMENSION,
        'optim_params':{
            'optim': 'AdamW',
            'beta1': beta1,
            'beta2': beta2,
            'weight_decay': decay,
            'learning_rate': lr,
        },
        'accumulation_iters': 1,
        'patch_size_mask' : 32,
        'mu': MU,
        'nu': NU,         
    },
)

In [11]:
import re
nums = [re.match(r'.*checkpoint_(.*).pth', x).group(1) for x in glob.glob(MODEL_SAVE_PATH+'*.pth')]
# if len(nums) > 0:
#     nums.remove("final")
nums = [int(x) for x in nums]

CHKPT = -1

if len(nums) != 0:
    CHKPT = max(nums)

    load_path = '{}_{}.pth'.format(MODEL_SAVE_PATH, CHKPT)
    chkpt = torch.load(load_path, map_location = {'cuda:1': device, 
                                                  'cuda:0': device})

    online_network.load_state_dict(chkpt['online_model_state_dict'])
    optim.load_state_dict(chkpt['optim_state_dict'])
    # lr_scheduler.load_state_dict(chkpt['scheduler_state_dict'])
    
    print(load_path)
    
    print("loaded earlier settings")

In [12]:
scaler = torch.cuda.amp.grad_scaler.GradScaler()
itm_loss_fn = torch.nn.BCEWithLogitsLoss()
online_network = online_network.to(DEVICE).train()

for epoch in range(CHKPT+1, EPOCHS + warmup_epochs):
    num_samples = 0
    pretrain_loss = 0
    # net all losses
    net_mim_loss = 0
    net_mim_joint_loss = 0
    net_mlm_loss = 0
    net_mlm_joint_loss = 0
    net_vicreg_image_loss = 0
    net_vicreg_text_loss = 0
    net_vicreg_joint_loss = 0
    net_itc_loss = 0
    net_itm_loss = 0
    for idx, data in (pbar := tqdm(enumerate(pretrain_dataloader), total = len(pretrain_dataloader))):
        img, img_mask, txt, attn_mask, masked_toks, masked_attn_mask, mask_indices = data
        
        # vision
        img = img.to(DEVICE)
        img_mask = img_mask.to(DEVICE)
        
        # language
        txt = txt.to(DEVICE)
        attn_mask = attn_mask.to(DEVICE)
        masked_toks = masked_toks.to(DEVICE)
        masked_attn_mask = masked_attn_mask.to(DEVICE)

        # indices for masked text: will be used for masked modeling
        mask_indices = mask_indices.float().to(DEVICE)
        
        # # masked image
        # masked_image = create_masked_image(img, img_mask)
        flattened_img_mask = img_mask.float().flatten(1)
        
        # create masks for joint representation modeling
        img_rep_masks = torch.cat([flattened_img_mask, torch.zeros_like(mask_indices)], axis = 1).unsqueeze(-1)
        txt_rep_masks = torch.cat([torch.zeros_like(flattened_img_mask), mask_indices], axis = 1).unsqueeze(-1)
        
        
        
        # masked modeling pretraining
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):                           # casting to f16
            # forward step for online network
            c_img_m_txt, m_img_c_txt, img_txt_joint, mask_img_rep, txt_prediction, img_rep, txt_rep = online_network(img,
                                                                                                      txt,
                                                                                                      attn_mask,
                                                                                                      image_text_matching = False,
                                                                                                      masked_pos = flattened_img_mask,
                                                                                                      masked_text = masked_toks)



            # MIM loss
            mim_loss = online_network.get_mim_loss(img_rep, img, flattened_img_mask)
            mim_loss_joint = online_network.get_joint_mim_loss(m_img_c_txt, img, flattened_img_mask)
            
            # MLM loss
            mlm_loss = online_network.get_mlm_loss(txt_prediction, txt, masked_toks)
            # mlm_loss_joint = online_network.get_joint_mlm_loss(c_img_m_txt, txt, masked_toks)
            
            # VICReg loss
            # vicreg_image = online_network.get_vicreg_loss(img_rep)
            # vicreg_txt = online_network.get_vicreg_loss(txt_rep)
            vicreg_joint = online_network.get_vicreg_loss(img_txt_joint)
            
            # ITC loss
            sim, itc_loss = online_network.get_itc_loss(img_rep, txt_rep)
            
            #itm loss
            # sample for each image and each text separately
            neg_txt, neg_img = online_network.get_samples(sim)
            
            itm_labels = torch.cat([torch.ones(len(img)),torch.zeros(2*len(img))],
                               dim=0).unsqueeze(1).float().to(DEVICE)
            # stack 
            itm_img_feats = torch.vstack([img_rep, img_rep[neg_img]])
            itm_txt_feats = torch.vstack([txt_rep[neg_txt], txt_rep])
            itm_txt_attn = torch.vstack([attn_mask[neg_txt], attn_mask])

            joint_rep_negs = online_network.fusion(itm_img_feats, itm_txt_feats, itm_txt_attn)['last_hidden_state']
            combined_reps = torch.vstack([img_txt_joint, joint_rep_negs])
            
            itm_outputs = online_network.itm__head(combined_reps[:, 0, :])
            
            
            # softmax probabilities
            itm_loss = itm_loss_fn(itm_outputs, itm_labels)
            
            # TOTAL LOSS
            net_loss = (mim_loss + mim_loss_joint) + \
                       (mlm_loss + mlm_loss_joint) + \
                       (vicreg_image + vicreg_txt + vicreg_joint) + \
                       (itc_loss) + (itm_loss)
            
        scaler.scale(net_loss).backward()
        scaler.unscale_(optim)
        torch.nn.utils.clip_grad_norm_(online_network.parameters(), 1.)
            
        # BACKPROP
        scaler.step(optim)        # fp16
        scaler.update()           # fp16
        optim.zero_grad(set_to_none = True)
        lr_scheduler.step_update(epoch * epoch_steps + idx)
        
        # update and calc loss
        num_samples+=1
        
        net_mim_loss+= mim_loss.item()
        net_mim_joint_loss+= mim_loss_joint.item()
        
        net_mlm_loss+= mlm_loss.item()
        net_mlm_joint_loss+= mlm_loss_joint.item()
        
        net_vicreg_image_loss+= vicreg_image.item()
        net_vicreg_text_loss+= vicreg_txt.item()
        net_vicreg_joint_loss+= vicreg_joint.item()
        
        net_itc_loss+= itc_loss.item()
        net_itm_loss+= itm_loss.item()
        pretrain_loss+= net_loss.item()
        pbar.set_description(f"Train Loss: {pretrain_loss/num_samples}")
        

    wandb.log({
            'epoch': epoch,
            'pretrain_loss': pretrain_loss/num_samples,
            'mim_loss': net_mim_loss/num_samples,
            'mim_joint_loss': net_mim_joint_loss/num_samples,
            'mlm_loss': net_mlm_loss/num_samples,
            'mlm_joint_loss': net_mlm_joint_loss/num_samples,

            'vicreg_image_loss': net_vicreg_image_loss/num_samples,
            'vicreg_text_loss': net_vicreg_text_loss/num_samples,
            'vicreg_joint_loss': net_vicreg_joint_loss/num_samples,

            'itc_loss': net_itc_loss/num_samples,
            'itm_loss': net_itm_loss/num_samples,
    })
    save_path = '{}_{}.pth'.format(MODEL_SAVE_PATH, epoch)
    torch.save(
            {
            'epoch': epoch,
            'online_model_state_dict': online_network.state_dict(),
            'optim_state_dict': optim.state_dict()
            },
        save_path
        )
    if (epoch-warmup_epochs+1) % 10 == 0:
        wandb.save(save_path)

Train Loss: 6.6459201762882945: 100%|██████████| 1511/1511 [12:52<00:00,  1.96it/s]


In [13]:
save_path = '{}_{}.pth'.format(MODEL_SAVE_PATH, 'final')
torch.save(online_network.state_dict(), save_path)
wandb.save(save_path)

['/home/ml/ML Projects/Denoising MAMO/wandb/run-20240421_105520-xjbbtpc2/files/Models/ViT-S BERT-S (fixed everything)/RegVLM - 1.0 - 1.0/checkpoint_final.pth']

In [14]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
wandb: ERROR Control-C detected -- Run data was not synced
